In [4]:
load_model = 'densenet_based_model2.pt'

data_dir = '../../../stanford_dogs_new/'

In [11]:
# import numpy as np

from IPython.display import Image

import os
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision.models as models
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

from torchvision.datasets import ImageFolder
from PIL import Image

from torch.autograd import Variable

from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy


from IPython.display import Image
from IPython.display import Markdown

from PIL import Image
import PIL.Image

import pandas as pd

# sklearn metrics
from sklearn import metrics
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import f1_score


## load the Densenet121 based model

In [6]:
densenet121_loaded = models.densenet121(pretrained=True)

for param in densenet121_loaded.parameters():
    param.requires_grad = False

    
densenet121_loaded.classifier = nn.Sequential(*list(densenet121_loaded.classifier.children())[:-1] + [nn.Linear(in_features=1024, out_features=120, bias=True)])

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
densenet121_loaded = densenet121_loaded.to(device)



densenet121_loaded.load_state_dict(torch.load('densenet121_based_model2.pt'))
densenet121_loaded.eval()

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [7]:
model = densenet121_loaded

In [8]:
# preprocess the image for the model
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
    )
])

In [9]:
path = '../../../stanford_dogs_new/test'

list_subfolders_with_paths = [f.path for f in os.scandir(path) if f.is_dir()]

# gets the breed name from subfolders name like 'n02099429-curly-coated_retriever'
def rename(name):
    return ' '.join(' '.join(name.split('-')[1:]).split('_'))

per_class_accuracy = dict()
total_accuracy = 0

y_true = []
y_pred = []

for breed_dir in list_subfolders_with_paths:
    
    breed = rename(breed_dir.split('/')[-1])
    
    images = os.listdir(breed_dir)
    
    matched = False
    matches = 0
    total_images = len(images)

    for image in images:

        input_image = Image.open(breed_dir + '/' + image)


        input_tensor = preprocess(input_image)
        if torch.cuda.is_available():
            input_tensor = Variable(input_tensor.cuda())

        input_batch = input_tensor.unsqueeze(0)
        out = model(input_batch)

        probabilities = torch.nn.functional.softmax(out[0], dim=0)
        # print(probabilities)

        with open("../../stanford_dogs_breeds_classes_final.txt", "r") as f:
            categories = [s.strip() for s in f.readlines()]

        predicted_breeds = []
        top1_prob, top1_catid = torch.topk(probabilities, 1)
        for i in range(top1_prob.size(0)):
            # predicted_breeds.append([categories[top3_catid[i]], top3_prob[i].item()*100])
            predicted_breed = categories[top1_catid[i]]
            y_true.append(breed)
            y_pred.append(predicted_breed)

        # list to be used directly by the application (predicted_breed, probability)
        # print("predicted_breeds are: \n", predicted_breed)
        if breed == predicted_breed:
            matched = True
            matches = matches + 1

        per_class_accuracy[breed] = matches / total_images * 100

In [12]:
df = pd.DataFrame(list(zip(y_true, y_pred)),
               columns =['Actual_breed', 'Predicted_breed'])

df

,Actual_breed,Predicted_breed
0,Pomeranian,Pomeranian
1,Pomeranian,Pomeranian
2,Pomeranian,Pomeranian
3,Pomeranian,Pomeranian
4,Pomeranian,Pomeranian
...,...,...
2105,papillon,papillon
2106,papillon,papillon
2107,papillon,papillon
2108,papillon,papillon


## Confusion Matrix

In [13]:
# Print the confusion matrix
print(metrics.confusion_matrix(y_true, y_pred))


[[23  0  0 ...  0  0  0]
 [ 0 17  0 ...  0  0  0]
 [ 0  0 21 ...  0  0  0]
 ...
 [ 0  0  0 ... 14  0  0]
 [ 0  0  0 ...  0 13  0]
 [ 0  0  0 ...  0  0 12]]


## Precision, Recall and F1

In [14]:
# Print the precision and recall, among other metrics
print(metrics.classification_report(y_true, y_pred, digits=3))

                                precision    recall  f1-score   support

                  Afghan hound      0.958     0.958     0.958        24
           African hunting dog      0.895     1.000     0.944        17
                      Airedale      0.913     1.000     0.955        21
American Staffordshire terrier      0.846     0.647     0.733        17
                   Appenzeller      0.600     0.375     0.462        16
            Australian terrier      0.929     0.650     0.765        20
            Bedlington terrier      0.950     1.000     0.974        19
          Bernese mountain dog      0.710     1.000     0.830        22
              Blenheim spaniel      0.900     0.947     0.923        19
                 Border collie      0.643     0.600     0.621        15
                Border terrier      0.857     1.000     0.923        18
                   Boston bull      0.857     0.947     0.900        19
          Bouvier des Flandres      1.000     0.733     0.846  

## Macro F1 Score

In [15]:
from sklearn.metrics import f1_score

f1_score(y_true, y_pred, average='macro')


0.8171115557459074

## Micro F1 Score

In [16]:
f1_score(y_true, y_pred, average='micro')


0.8251184834123223

## Weighted F1 Score

In [17]:
f1_score(y_true, y_pred, average='weighted')

0.8201371780044038

## Cohen Kappa Score

In [18]:
from sklearn.metrics import cohen_kappa_score

cohen_kappa_score(y_true, y_pred)

0.8236130074957506

## Total Accuracy calculated from per class accuracy

In [19]:
# calculated from per class accuracy
total_accuracy = sum(per_class_accuracy.values())/120
total_accuracy

81.99398446196048

## Per Breed Accuracy

### Top 10 Breeds with least accuracy

In [20]:
k = 10
sorted(per_class_accuracy.items(), key=lambda x: x[1])[:k]

[('Eskimo dog', 13.333333333333334),
 ('Appenzeller', 37.5),
 ('Walker hound', 37.5),
 ('collie', 37.5),
 ('standard schnauzer', 43.75),
 ('Yorkshire terrier', 47.05882352941176),
 ('Lhasa', 47.368421052631575),
 ('Lakeland terrier', 50.0),
 ('miniature poodle', 50.0),
 ('boxer', 56.25)]

### Top 10 Breeds with most accuracy

In [22]:
k = 10
sorted(per_class_accuracy.items(), key=lambda x: x[1], reverse=True)[:k]

[('Japanese spaniel', 100.0),
 ('Irish setter', 100.0),
 ('German short haired pointer', 100.0),
 ('Dandie Dinmont', 100.0),
 ('clumber', 100.0),
 ('chow', 100.0),
 ('Old English sheepdog', 100.0),
 ('miniature schnauzer', 100.0),
 ('keeshond', 100.0),
 ('Samoyed', 100.0)]